In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import csv

from sklearn.cluster import MiniBatchKMeans
# from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import glob
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import metrics
from commonFunctions import *

In [2]:
def shadow_removalDialate(img):
    # img2 = cv2.imread(f'../Dataset_0-5/men/{num}/4_men (49).JPG')
    img2 = cv2.resize(img, (256, 256))
    
    img_hls = cv2.cvtColor(img2, cv2.COLOR_BGR2HLS)

    # define lower and upper bounds for blue color in HLS format
    lower_blue = np.array([0, 0, 60])
    upper_blue = np.array([20, 255, 255])

    # create a mask for blue color in HLS format
    mask = cv2.inRange(img_hls, lower_blue, upper_blue)
    # daialation
    kernel = np.ones((3,3),np.uint8)
    mask = cv2.dilate(mask,kernel,iterations = 8)
    # apply the mask to the original image
    result = cv2.bitwise_and(img2, img2, mask=mask)
    return result

In [3]:
def preprocessing(img,img2):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply histogram equalization to enhance contrast and remove lighting effects
    equalized = cv2.equalizeHist(gray)
    io.imshow(equalized)
    # # Find the contours of the binary image
    contours, hierarchy = cv2.findContours(equalized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_contour = max(contours, key=cv2.contourArea)

    # # Draw the largest contour on the original image
    cv2.drawContours(img2, [max_contour], 0, (0, 255, 0), 2)
    io.imshow(img2)
    # # Show the image with the largest contour drawn
    # # Create a bounding box around the hand
    x, y, w, h = cv2.boundingRect(max_contour)

    # # Crop the image to the bounding box around the hand
    hand = equalized[y:y+h, x:x+w]
    img3 = cv2.resize(hand, (256, 256))

    return img3

In [4]:
arr = ["0","1","2","3","4","5"]
dico = []


for label in arr:
    dirList = glob.glob("../Dataset_0-5/men/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            hand = shadow_removalDialate(im)
            result = preprocessing(hand,im)
                   
            surf = cv2.xfeatures2d.SURF_create()
            kp, des = surf.detectAndCompute(result, None)
            for d in des:
                dico.append(d)
        except:
            pass
        

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=120)
kmeans.fit(dico)
joblib.dump(kmeans, "model/surfkmeans.pkl")

In [ ]:
histo_list = []
Y = []
arr = ["0","1","2","3","4","5"]

lower_blue = np.array([0, 0, 65])
upper_blue = np.array([20, 255, 255])


for label in arr:
    dirList = glob.glob("../Dataset_0-5/men/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            hand = shadow_removalDialate(im)
            result = preprocessing(hand,im)
                   
            surf = cv2.xfeatures2d.SURF_create()
            kp, des = surf.detectAndCompute(result, None)
            histo = np.zeros(120)
            nkp = np.size(kp)
            for d in des:
                idx = kmeans.predict([d])
                histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

            histo_list.append(histo)
            Y.append(int(label))
        except:
            pass

In [ ]:
X = np.array(histo_list)
train_x,test_x,train_y,test_y = train_test_split(histo_list,Y, test_size = 0.2)
classifier=SVC(kernel="poly", random_state=6, degree=6)
classifier.fit(train_x,train_y)
joblib.dump(classifier, "model/surf_model.pkl")

In [ ]:
from sklearn import metrics

prediction=classifier.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))